In [1]:

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import os
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

# from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [2]:
import os
os.chdir("/home/earapidis/Desktop/BinaryNet.pytorch")
os.getcwd()

'/home/earapidis/Desktop/BinaryNet.pytorch'

In [3]:
from models.binarized_modules import  BinarizeLinear,BinarizeConv2d, binarized
from models.binarized_modules import  Binarize,HingeLoss


In [4]:
cuda = True

In [5]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [6]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [7]:
test_batch_size=1000

In [8]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=False, **kwargs)

In [9]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.fc1 = BinarizeLinear(784, 512)
#         self.bn1 = nn.BatchNorm1d(512)
#         self.htanh1 = nn.Hardtanh()

#         self.fc2 = BinarizeLinear(512, 64)
#         self.bn2 = nn.BatchNorm1d(64)
#         self.htanh2 = nn.Hardtanh()

        
#         self.fc3 =BinarizeLinear(64,10)

#         self.drop=nn.Dropout(0.1)

#     def forward(self, x):
#         x = x.view(-1, 28*28)
#         x = self.fc1(x)
#         x = self.bn1(x)
#         x = self.htanh1(x)

#         x = self.drop(x)

#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = self.htanh2(x)
        
#         x = self.drop(x)

#         x = self.fc3(x)
#         return x
#         # return self.logsoftmax(x)

# model = Net()
# if cuda:
#     torch.cuda.set_device(0)
#     model.cuda()


In [9]:
from mnist_bnn import Net
# class Net(nn.Module):

#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.fc1 = BinarizeLinear(784, 512)
#         self.bn1 = nn.BatchNorm1d(512)
#         self.htanh1 = nn.Hardtanh()

#         self.fc2 = BinarizeLinear(512, 32)
#         self.bn2 = nn.BatchNorm1d(32)
#         self.htanh2 = nn.Hardtanh()

        
#         self.fc3 =BinarizeLinear(32,10)

#         self.drop=nn.Dropout(0.1)

#     def forward(self, x):
#         x = x.view(-1, 28*28)
#         x = self.fc1(x)
#         x = self.bn1(x)
#         x = self.htanh1(x)

#         x = self.drop(x)

#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = self.htanh2(x)
        
#         x = self.drop(x)

#         x = self.fc3(x)
#         return x
#         # return self.logsoftmax(x)

model = Net()
if cuda:
    torch.cuda.set_device(0)
    model.cuda()


In [10]:
lr=0.001

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [12]:
log_interval=10

In [13]:
def test(model ):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    acc = 100. * correct / len(test_loader.dataset)
    return acc

In [14]:
model_sim = 4

save_dir = os.path.abspath("/shares/bulk/earapidis/saved_models")
models_path = os.path.join(save_dir,f"model_{model_sim}")

In [15]:
# model_path = os.path.join(models_path,f"epoch_7.pth")
model_path = os.path.join(models_path,f"best.pth")
model = Net()
model.load_state_dict(torch.load(model_path))
if cuda:
    torch.cuda.set_device(0)
    model.cuda()

In [16]:
model.state_dict

<bound method Module.state_dict of Net(
  (fc1): BinarizeLinear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=512, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinear(in_features=32, out_features=10, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
)>

In [17]:
test(model)


Test set: Average loss: 0.0002, Accuracy: 9597/10000 (96%)



tensor(95.9700)

In [18]:
from collections import defaultdict
activations = defaultdict(list)


# Hook function to capture activations
def hook_fn_fc(module, input, output,activations,name):
    activations[name].append(output.clone().detach().cpu())
    # Append the activations of the current batch
    # activations_fc3.append(output.clone().detach().cpu())

# target_layers = ["htanh1","htanh2","fc2","fc3","fc1"]
target_layers = ["htanh1","htanh2","fc2","fc3","fc1"]
hooks = []
for name, module in model.named_modules():
    if name in target_layers:
        hook = module.register_forward_hook(lambda module,input,output,name=name : hook_fn_fc(module,input,output,activations,name=name))
        hooks.append(hook)
        print(name,module)

targets = []
predictions = []
# Process the dataset in batches
correct = 0
binarized_inputs = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        predictions.append(pred.detach().cpu())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        # break
        targets.append(target.cpu())
        copy_data = data.clone().detach().cpu()
        copy_data=copy_data.view(test_batch_size,28*28)

        bin_data = binarized(copy_data)
        
        activations["bin_input"].append(bin_data)
        # binarized_inputs.

# Remove the hook after processing
for hook in hooks:
    hook.remove()


fc1 BinarizeLinear(in_features=784, out_features=512, bias=True)
htanh1 Hardtanh(min_val=-1.0, max_val=1.0)
fc2 BinarizeLinear(in_features=512, out_features=32, bias=True)
htanh2 Hardtanh(min_val=-1.0, max_val=1.0)
fc3 BinarizeLinear(in_features=32, out_features=10, bias=True)


In [21]:
activations.keys()

dict_keys(['fc1', 'htanh1', 'fc2', 'htanh2', 'fc3', 'bin_input'])

In [24]:
activations["fc3"][0].shape

torch.Size([1000, 10])

In [25]:
for key in activations.keys():
    print(f"module {key}, batches : {len(activations[key])}, batch size : {activations[key][0].shape[0]}, one row size: {activations[key][0].shape[1]}")

module fc1, batches : 10, batch size : 1000, one row size: 512
module htanh1, batches : 10, batch size : 1000, one row size: 512
module fc2, batches : 10, batch size : 1000, one row size: 32
module htanh2, batches : 10, batch size : 1000, one row size: 32
module fc3, batches : 10, batch size : 1000, one row size: 10
module bin_input, batches : 10, batch size : 1000, one row size: 784


In [26]:
activations["bin_input"][0].shape

torch.Size([1000, 784])

In [27]:
save_sim_data = os.path.abspath("/shares/bulk/earapidis/sim_data")
save_sim_data = os.path.join(save_sim_data,f"model_{model_sim}")
print(save_sim_data)
os.makedirs(save_sim_data,exist_ok=True)

/shares/bulk/earapidis/sim_data/model_4


In [28]:
model.bn1.running_mean.shape

torch.Size([512])

In [29]:
def get_layer_num(layer_name):
    layer_num=""
    if layer_name.endswith("1"):
        layer_num = "layer_1"
    if layer_name.endswith("2"):
        layer_num = "layer_2"
    if layer_name.endswith("3"):
        layer_num = "layer_3"
    return layer_num

In [30]:
names = defaultdict(dict)
for name,layer in model.named_modules():
    foo = ""
    layer_num = ""
    if name.startswith("fc"):
        foo = "fc"
    elif name.startswith("bn"):
        foo = "bn"
    elif name.startswith("htanh"):
        foo = "activation"
    
    layer_num = get_layer_num(name)
    
    if foo!="" and layer_num!="":
        names[layer_num][foo]=name
names


defaultdict(dict,
            {'layer_1': {'fc': 'fc1', 'bn': 'bn1', 'activation': 'htanh1'},
             'layer_2': {'fc': 'fc2', 'bn': 'bn2', 'activation': 'htanh2'},
             'layer_3': {'fc': 'fc3'}})

In [31]:
model_dict = dict(names)
model_dict["path"] = model_path
model_dict

{'layer_1': {'fc': 'fc1', 'bn': 'bn1', 'activation': 'htanh1'},
 'layer_2': {'fc': 'fc2', 'bn': 'bn2', 'activation': 'htanh2'},
 'layer_3': {'fc': 'fc3'},
 'path': '/shares/bulk/earapidis/saved_models/model_4/best.pth'}

In [33]:
sim_data_dict = defaultdict(dict)

for layer_name,values in activations.items():
    if layer_name!="bin_input":
        layer_path = os.path.join(save_sim_data,layer_name)
        print(layer_path)
        os.makedirs(layer_path,exist_ok=True)

        if "fc" in layer_name:
            binary_data = torch.stack(values, dim=0)
        else:
            binary_data = binarized(torch.stack(values, dim=0))

        layer_num=get_layer_num(layer_name)
        
        binary_data = binary_data.view(-1,binary_data.shape[-1])
        path = os.path.join(layer_path,"binary_data.pt")
        sim_data_dict[layer_num][layer_name]={"output":path}
        torch.save(binary_data,path)
        print(binary_data.shape)
    else:
        binary_data = activations["bin_input"]
        binary_data = torch.stack(values, dim=0)
        binary_data = binary_data.view(-1,binary_data.shape[-1])
        layer_path = os.path.join(save_sim_data,"bin_input")
        print(layer_path)
        print(binary_data.shape)

        os.makedirs(layer_path,exist_ok=True)

        path = os.path.join(layer_path,"binary_data.pt")
        sim_data_dict["layer_1"]["bin_input"]={"input":path}

    # module_layer = getattr(model, layer_name)
    # weights = binarized(module_layer.weight.detach().cpu())  # Weights tensor
    # biases = module_layer.bias.detach().cpu() if module_layer.bias is not None else None  # Bias tensor
    # print(weights.shape)
    # print(biases.shape)
    # break
sim_data_dict
    

/shares/bulk/earapidis/sim_data/model_4/fc1
torch.Size([10000, 512])
/shares/bulk/earapidis/sim_data/model_4/htanh1
torch.Size([10000, 512])
/shares/bulk/earapidis/sim_data/model_4/fc2
torch.Size([10000, 32])
/shares/bulk/earapidis/sim_data/model_4/htanh2
torch.Size([10000, 32])
/shares/bulk/earapidis/sim_data/model_4/fc3
torch.Size([10000, 10])
/shares/bulk/earapidis/sim_data/model_4/bin_input
torch.Size([10000, 784])


defaultdict(dict,
            {'layer_1': {'fc1': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.pt'},
              'htanh1': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.pt'},
              'bin_input': {'input': '/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.pt'}},
             'layer_2': {'fc2': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.pt'},
              'htanh2': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh2/binary_data.pt'}},
             'layer_3': {'fc3': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc3/binary_data.pt'}}})

In [34]:
for layer_name in activations.keys():
    if layer_name!="bin_input":
        layer = getattr(model, layer_name)
        layer_path = os.path.join(save_sim_data,layer_name)
        if hasattr(layer,"weight"):
            weights = binarized(layer.weight.detach().cpu())  # Weights tensor
            path = os.path.join(layer_path,"weights.pt")

            torch.save(weights,path)
            print(f"{layer_name} weights: {weights.shape}")

            layer_num = get_layer_num(layer_name)
            if layer_num!="":
                sim_data_dict[layer_num][layer_name]["weights"]=path

        if hasattr(layer,"bias"):
            if layer.bias is not None:
                bias = layer.bias.detach().cpu() if layer.bias is not None else None  # Bias tensor
                path = os.path.join(layer_path,"bias.pt")
                sim_data_dict[layer_num][layer_name]["bias"]=path

                torch.save(bias,path)
                print(layer_name,f"bias : {bias.shape}")
  

fc1 weights: torch.Size([512, 784])
fc1 bias : torch.Size([512])
fc2 weights: torch.Size([32, 512])
fc2 bias : torch.Size([32])
fc3 weights: torch.Size([10, 32])
fc3 bias : torch.Size([10])


In [35]:
sim_data_dict["path"] = model_dict["path"]
sim_data_dict = dict(sim_data_dict)


In [36]:
predictions = torch.stack(predictions, dim=0)
predictions = predictions.squeeze(-1)
predictions = predictions.view(-1)
predictions.shape

torch.Size([10000])

In [37]:
targets = torch.stack(targets,dim=0)
targets = targets.view(-1)
targets.shape

torch.Size([10000])

In [38]:
path = os.path.join(save_sim_data,"targets.pt")
sim_data_dict["targets"] = path
torch.save(targets,path)

path = os.path.join(save_sim_data,"predictions.pt")
sim_data_dict["predictions"] = path
torch.save(predictions,path)

In [39]:
htanh2 = torch.load(os.path.join(save_sim_data,"htanh2","binary_data.pt"))
htanh1 = torch.load(os.path.join(save_sim_data,"htanh1","binary_data.pt"))

In [40]:
index = 7688

In [41]:
one_input = htanh1[index]
one_input.shape

torch.Size([512])

In [42]:
one_input = one_input.unsqueeze(0)

one_input.shape

torch.Size([1, 512])

In [43]:
model.eval()  # Set the model to evaluation mode
model.cpu()
with torch.no_grad():  # Disable gradient computation for evaluation
        one_input= Variable(one_input)
        output_fc2 = model.fc2(one_input)
        output_bn2 = model.bn2(output_fc2)
        output_htanh2 = model.htanh2(output_bn2)
b_output_htanh2 = binarized(output_htanh2)
b_output_htanh2 = b_output_htanh2.squeeze()

In [44]:
type(output_htanh2)

torch.Tensor

In [45]:
output_htanh2

tensor([[-1.0000e+00,  7.1902e-01,  1.0000e+00, -1.0000e+00, -6.5736e-01,
         -1.5793e-03,  7.1636e-03, -1.2026e-02,  9.8564e-01,  6.6669e-04,
          5.9626e-01, -2.6584e-01,  1.0000e+00,  9.7218e-03, -1.0000e+00,
         -1.0000e+00, -4.8500e-02, -8.3804e-04, -3.0977e-01,  8.3901e-01,
         -5.9843e-04, -1.0000e+00, -1.0000e+00, -9.1583e-04, -2.8063e-03,
         -1.8865e-01,  3.7032e-01,  7.4953e-01,  1.7832e-01, -2.4491e-03,
          4.4013e-01,  7.6972e-01]])

In [46]:
htanh2_hook = htanh2[index]


In [47]:
torch.equal(b_output_htanh2,htanh2_hook)

True

In [48]:
model

Net(
  (fc1): BinarizeLinear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=512, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinear(in_features=32, out_features=10, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
)

In [49]:
import yaml
with open('model_structure.yml', 'w') as outfile:
    yaml.dump(model_dict, outfile, default_flow_style=False)

In [50]:
sim_data_dict

{'layer_1': {'fc1': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc1/weights.pt',
   'bias': '/shares/bulk/earapidis/sim_data/model_4/fc1/bias.pt'},
  'htanh1': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.pt'},
  'bin_input': {'input': '/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.pt'}},
 'layer_2': {'fc2': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc2/weights.pt',
   'bias': '/shares/bulk/earapidis/sim_data/model_4/fc2/bias.pt'},
  'htanh2': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh2/binary_data.pt'}},
 'layer_3': {'fc3': {'output': '/shares/bulk/earapidis/sim_data/model_4/fc3/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc3/weights.pt',
   'bias': '/shares/bulk/earapidis/sim_data/model_4/fc3/bias.pt'}},
 'path': '/shares/bulk/earapi

In [51]:
sim_data_dict = dict(sim_data_dict)
with open('sim_data_structure.yml', 'w') as outfile:
    yaml.dump(sim_data_dict, outfile, default_flow_style=False)

In [52]:
with open("/home/earapidis/Desktop/BinaryNet.pytorch/model_structure.yml", 'r') as file:
        data = yaml.safe_load(file)
data

{'layer_1': {'activation': 'htanh1', 'bn': 'bn1', 'fc': 'fc1'},
 'layer_2': {'activation': 'htanh2', 'bn': 'bn2', 'fc': 'fc2'},
 'layer_3': {'fc': 'fc3'},
 'path': '/shares/bulk/earapidis/saved_models/model_4/best.pth'}

In [53]:
with open("/home/earapidis/Desktop/BinaryNet.pytorch/sim_data_structure.yml", 'r') as file:
    sim_data = yaml.safe_load(file)
sim_data

{'layer_1': {'bin_input': {'input': '/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.pt'},
  'fc1': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc1/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc1/weights.pt'},
  'htanh1': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.pt'}},
 'layer_2': {'fc2': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc2/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc2/weights.pt'},
  'htanh2': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh2/binary_data.pt'}},
 'layer_3': {'fc3': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc3/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc3/binary_data.pt',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc3/weights.pt'}},
 'path': '/shares/bulk/earapi

In [54]:
import copy
import h5py
sim_data_h5 = copy.deepcopy(sim_data)

for layer, values in sim_data.items():
    if "layer" in layer:
        for name, val in values.items():
            print(name)
            
            for metric, metric_path in val.items():
                print(metric)
                print(metric_path)
                if metric=="output" or metric=="input":

                    file_name = os.path.split(metric_path)[-1].split(".")[0]
                    print(file_name)
                    par_dir = os.path.dirname(metric_path)
                    print(par_dir)
                
                    h5_path = os.path.join(par_dir,f"{file_name}.h5")
                    sim_data_h5[layer][name][metric] = h5_path
                    print(h5_path)
                    print()
                    tensor_data = torch.load(metric_path)
                    with h5py.File(h5_path, "w") as f:
                        f.create_dataset("my_tensor", data=tensor_data.numpy())  # Save as NumPy array
    if layer=="predictions" or layer=="targets":
        tensor_data = torch.load(values)
        par_dir = os.path.dirname(values)
        file_name = os.path.split(values)[-1].split(".")[0]

        print(par_dir)
    
        h5_path = os.path.join(par_dir,f"{file_name}.h5")
        sim_data_h5[layer] = h5_path
        print(h5_path)
        print()
        with h5py.File(h5_path, "w") as f:
            f.create_dataset("my_tensor", data=tensor_data.numpy())  # Save as NumPy array

bin_input
input
/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.pt
binary_data
/shares/bulk/earapidis/sim_data/model_4/bin_input
/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.h5

fc1
bias
/shares/bulk/earapidis/sim_data/model_4/fc1/bias.pt
output
/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.pt
binary_data
/shares/bulk/earapidis/sim_data/model_4/fc1
/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.h5

weights
/shares/bulk/earapidis/sim_data/model_4/fc1/weights.pt
htanh1
output
/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.pt
binary_data
/shares/bulk/earapidis/sim_data/model_4/htanh1
/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.h5

fc2
bias
/shares/bulk/earapidis/sim_data/model_4/fc2/bias.pt
output
/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.pt
binary_data
/shares/bulk/earapidis/sim_data/model_4/fc2
/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.h5

weights
/shares/bulk/earapidis/sim_data/mode

In [55]:
sim_data_h5

{'layer_1': {'bin_input': {'input': '/shares/bulk/earapidis/sim_data/model_4/bin_input/binary_data.h5'},
  'fc1': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc1/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc1/binary_data.h5',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc1/weights.pt'},
  'htanh1': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh1/binary_data.h5'}},
 'layer_2': {'fc2': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc2/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc2/binary_data.h5',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc2/weights.pt'},
  'htanh2': {'output': '/shares/bulk/earapidis/sim_data/model_4/htanh2/binary_data.h5'}},
 'layer_3': {'fc3': {'bias': '/shares/bulk/earapidis/sim_data/model_4/fc3/bias.pt',
   'output': '/shares/bulk/earapidis/sim_data/model_4/fc3/binary_data.h5',
   'weights': '/shares/bulk/earapidis/sim_data/model_4/fc3/weights.pt'}},
 'path': '/shares/bulk/earapi

In [56]:
with open('sim_data_structure_h5.yml', 'w') as outfile:
    yaml.dump(sim_data_h5, outfile, default_flow_style=False)